In [7]:
from bokeh.resources import INLINE
import bokeh.io

bokeh.io.output_notebook(INLINE)

Loading BokehJS ...

In [8]:
import numpy as np
import pandas as pd
import scipy.optimize
import scipy.stats as st
import scipy.special

import bokeh.io
bokeh.io.output_notebook()
import panel as pn
pn.extension()

def style(p, autohide=False):
    p.title.text_font="Helvetica"
    p.title.text_font_size="16px"
    p.title.align="center"
    p.xaxis.axis_label_text_font="Helvetica"
    p.yaxis.axis_label_text_font="Helvetica"
    
    p.xaxis.axis_label_text_font_size="13px"
    p.yaxis.axis_label_text_font_size="13px"
    p.background_fill_alpha = 0
    if autohide: p.toolbar.autohide=True
    return p

Loading BokehJS ...

<img src="__150.4.2abc.jpg">
<img src="__150.4.2d.jpg">
<img src="__150.4.2e.jpg">

# f)
# General Knowledge
Some considerations for our simulations before the formal discussion:
- Even though these images from the paper suggest that $k_p$ should not be the same between agonist and endogenous M (they seem to have different recruiting powers and more complex mechanics involved, primarily the role of CD4/8), I will keep k_p to be the same and mainly focus on the effects of differing $k_{-1}$'s between agonist and endogenous complexes. [Bionumbers](https://bionumbers.hms.harvard.edu/files/Comparison%20of%20kinetic%20constants%20for%20phosphorylation%20by%20protein%20kinase%20C.pdf) has this at around 1-10 inverse seconds. 
><img src="figure3.png" width=90%>

>*Shown are the four states of phosphorylation described in this review, based on phosphorylation of Tyr394 in the activation loop of the catalytic domain and of Tyr505 in the negative regulatory segment. Also shown are the molecules thought to regulate the transition between the different phosphorylation states.*

><img src="figure4.png" width=90%>

>*In the basal state, the ζ-chain ITAMs are phosphorylated (red dots) and bound to autoinhibited Zap70. Lck initiates activation by phosphorylating Zap70 Tyr319 in interdomain B, relieving Zap70 autoinhibition and creating a binding site for the Lck SH2 domain, further stabilizing the activated state of Lck and, thereby, initiating a positive feedback loop in which Lck can promote further local phosphorylation events. Zap70 trans-autophosphorylation of its activation loop results in its catalytic activation.*

- Note that [$M_{\mathrm{endogenous}}$] is waaay higher than [$M_{\mathrm{agonist}}$]. We will use log-scales for initial Mo's going forward. 
- Uri Alon's book says that the dissociation constant ($k_{\mathrm{off}}$ / $k_{\mathrm{on}}$) for the initial complexes formed by endo (short for endogenous), and agon (short for agonist) is experimentally found to be at most a ten-fold difference. 
- Intuitively, we recognize that adding these kinetic phosphorylative steps will introduce this aspect of selectivity, where we can further distinguish between agonist and endogenous MHC. However, by looking at the functional form of $C_n$, we see that higher $n$ means the $\alpha$ term (which is less than one) will get exponentiated, meaning decreased sensitivity to $[M_o]$. We will examine how $n$ and $k_{-1}$ can work together to optimize selectivity without losing sensitivity. 

# Dynamical Studies 💫
We start by looking at our trajectories. This might not be particularly insightful or tell us anything we don't already know (after all that hard work to find steady state behaviors), but I think I'll have a firmer grasp for which parameter schemes to focus on, given the huge dimensionality of the space we're exploring. This will also give me some insight into $k_n$, and what it does for our system.

I will fix $n$ to 10 bc I am short on time and could not get the arrays/sliders to work smoothly, but we will return to this parameter shortly.

In [9]:
# .... DERIVATIVES .... 
def derivs(concs, t, k_on, k_off, k_p, k_1, k_n):
    M, T, C0, C1, C2, C3, C4, C5, C6, C7, C8, C9, Cn = concs
    
    Ci_sum = C1 + C2 + C3 + C4 + C5 + C6 + C7 + C8 + C9
    deriv_Cm = -k_on*M*T + k_off*C0 + k_1*Ci_sum + k_n*Cn
    deriv_Ct = deriv_Cm
    deriv_C0 = k_on*M*T - k_off*C0 - k_p*C0
    deriv_C1 = k_p*C0 - (k_p+k_1)*C1
    deriv_C2 = k_p*C1 - (k_p+k_1)*C2
    deriv_C3 = k_p*C2 - (k_p+k_1)*C3
    deriv_C4 = k_p*C3 - (k_p+k_1)*C4
    deriv_C5 = k_p*C4 - (k_p+k_1)*C5
    deriv_C6 = k_p*C5 - (k_p+k_1)*C6
    deriv_C7 = k_p*C6 - (k_p+k_1)*C7
    deriv_C8 = k_p*C7 - (k_p+k_1)*C8
    deriv_C9 = k_p*C8 - (k_p+k_1)*C9
    deriv_Cn = k_p*C9 - k_n*Cn
    
    return np.array(
        [deriv_Cm,
         deriv_Ct,
         deriv_C0,
         deriv_C1,
         deriv_C2,
         deriv_C3,
         deriv_C4,
         deriv_C5,
         deriv_C6,
         deriv_C7,
         deriv_C8,
         deriv_C9, 
         deriv_Cn,   
        ])

In [10]:
# .... COLORS ....
T_color = "#a54a5c"
Cn_color = "#74a0b2"
palette = [T_color,
           '#0f1524',
           '#2f3958',
           '#4e5980',
           '#6e79a8',
           '#868dbf',
           '#9693c3',
           '#a599c6',
           '#b39fc9',
           '#bea6cb',
           '#caadcd',
           Cn_color
          ]

# .... WIDGETS ....
log_Mo_slider = pn.widgets.FloatSlider(start=-2, end=6, value=2, step=0.05, name="log Mo", width=150)
log_Kd_slider = pn.widgets.FloatSlider(start=-5, end=1, value=-3, step=0.1, name="log Kd", width=150)
log_k_p_slider = pn.widgets.FloatSlider(start=0.75, end=1.25, value=1, step=0.01, name="log k_p", width=150)
log_k_1_slider = pn.widgets.FloatSlider(start=-3, end=1, value=-2, step=0.01, name="log k_1", width=150)
log_k_n_slider = pn.widgets.FloatSlider(start=-5, end=2, value=-2, step=0.01, name="log k_n", width=150)
t_max_slider = pn.widgets.FloatSlider(start=1, end=10, value=3, step=0.1, name="t maximum", width=275)

# .... DASHBOARD ....
@pn.depends(log_Mo_slider.param.value,
            log_Kd_slider.param.value,
            log_k_p_slider.param.value, 
            log_k_1_slider.param.value, 
            log_k_n_slider.param.value, 
            t_max_slider.param.value
           )
def dynamic_plotter(log_Mo, log_Kd, log_k_p, log_k_1, log_k_n, t_max):
    To = 1
    Mo = np.power(10.0, log_Mo)
    Kd = np.power(10.0, log_Kd)
    k_on, k_off = 1, Kd
    k_p, k_1, k_n = np.power(10.0, log_k_p), np.power(10.0, log_k_1), np.power(10.0, log_k_n)
    concs_init = np.array([Mo, To, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
    args=(k_on, k_off, k_p, k_1, k_n)

    t = np.linspace(0, t_max, 1000)
    _trajectories = scipy.integrate.odeint(derivs, concs_init, t, args=args)
    M, T, C0, C1, C2, C3, C4, C5, C6, C7, C8, C9, Cn = _trajectories.T

    plot_trajectories = [T, C0, C1, C2, C3, C4, C5, C6, C7, C8, C9, Cn]
    labels = ['T', 'C0', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'Cn']
    norm_arr = [False, False, True, True, True, True, True, True, True, True, True, False]
    
    p = bokeh.plotting.figure(height=400, width=600, title="Dynamical Studies: n=10",
                         x_axis_label="time", y_axis_label="concentrations")
    for arr, label, color, norm in zip(plot_trajectories, labels, palette, norm_arr):
#         if norm: arr = arr/2/arr.max()
        p.line(t, arr, legend_label=label, line_width=2.5, line_color=color)
    p.legend.click_policy="hide"
    
    return style(p)

lay_widgets = pn.Row(pn.Column(log_Mo_slider, align="center"), 
                     pn.Column(log_Kd_slider, log_k_p_slider), 
                     pn.Column(log_k_1_slider, log_k_n_slider), 
                     align="center")
lay_time = pn.Row(t_max_slider, align="center")
dashboard_trajectories = pn.Column(lay_widgets, dynamic_plotter, lay_time, align="center")
dashboard_trajectories

Column(align='center')
    [0] Row(align='center')
        [0] Column(align='center')
            [0] FloatSlider(end=6, name='log Mo', start=-2, step=0.05, value=2, width=150)
        [1] Column
            [0] FloatSlider(name='log Kd', start=-5, value=-3, width=150)
            [1] FloatSlider(end=1.25, name='log k_p', start=0.75, step=0.01, value=1, width=150)
        [2] Column
            [0] FloatSlider(name='log k_1', start=-3, step=0.01, value=-2, width=150)
            [1] FloatSlider(end=2, name='log k_n', start=-5, step=0.01, value=-2, width=150)
    [1] ParamFunction(function)
    [2] Row(align='center')
        [0] FloatSlider(end=10, name='t maximum', start=1, value=3, width=275)

In [14]:
# if the above does not work
dashboard_trajectories.show()

Launching server at http://localhost:49424


### observations
- look at Cn rise!
- increasing Mo from 1e-2 to 1e2 demonstrates sensitivity to Mo; past a certain point doesn't change much, for this system's relative concentrations and time-scales will then be driven by availability of T. However, when comparing fractions and selectivity, the M concentrations become incredibly important.
- increasing $K_d$ introduces delay (the rate at which initial complex falls off), this kinetically makes sense since the half time for first-order dissociative processes is an inverse function of $k_{\mathrm{off}}$
- faster phosphorylation $k_p$ leads to faster formation of Cn
- decreasing $k_{-1}$ (a stickier reaction, less fall-back) beyond a certain threshold ceases to affect our system, $k_{-1}$ too high means our reaction is not sticky enough, and Cn is not being formed at all, and Co just kinda sits there (this can be seen analytically as well.)
- **the coolest widget is log k_n** (increasing t maximum helps). It shows us how we can tune the responsiveness of Cn. Intuitively, the endogenous $k_n$ should not be too different from $k_{-1}$, but for agonist complexes, the final form should be incredibly stabilizing. It requires some playing-around with, but after seeing how faster $k_{-1}$'s operate and send our Cn's to zero, we see that $k_n$ can manage to recover $C_n$ (although, the slopes will get flatter since this process will be slow.)

# Steady State Analysis

In [11]:
# FIXED
k_on = 1        # first dissociation
k_p = 1e1

Kd_agon = 1       
Kd_endo = 10    # endogenous more likely to dissociate 10-fold

k_off_agon = Kd_agon
k_off_endo = Kd_endo

T = 1


# TUNABLE
n = 10          # number of phosphorylation steps
      
k_1_agon = 1    # fall off agonist
k_1_endo = 0.1  # fall off endogenous

M_agon = 1
M_endo = 1e2


# DEPENDENT
alpha_agon = 1/(1 + k_1_agon/k_p)
alpha_endo = 1/(1 + k_1_endo/k_p)
kappa_agon = (1-alpha_agon)*(Kd_agon + k_p/k_on)
kappa_endo = (1-alpha_endo)*(Kd_endo + k_p/k_on)


# steady state functions for retrieval later
Cn_agon = (alpha_agon**n)/(1-alpha_agon) * M_agon*T/(Kd_agon + k_p/k_on)
Cn_endo = (alpha_endo**n)/(1-alpha_endo) * M_endo*T/(Kd_endo + k_p/k_on)

Ctot_agon = M_agon*T/(Kd_agon + k_p/k_on) / (1-alpha_agon)
Ctot_endo = M_endo*T/(Kd_endo + k_p/k_on) / (1-alpha_endo)

In [15]:
red = "#ce554e"
green = "#2ea58e"
blue = "#4d53b5"


log_k_p_slider = pn.widgets.FloatSlider(
    name="log kp", start=0, end=2, step=0.01, value=1, width=150)
log_k_1_agon_slider = pn.widgets.FloatSlider(
    name="agon log k1", start=-3, end=1, step=0.01, value=-2, width=150)
log_k_1_endo_slider = pn.widgets.FloatSlider(
    name="endo log k1", start=-3, end=1, step=0.01, value=-1, width=150)
log_M_agon_slider = pn.widgets.FloatSlider(
    name="agon log Mo", start=0, end=1, step=0.01, value=0, width=150)
log_M_endo_slider = pn.widgets.FloatSlider(
    name="endo log Mo", start=1, end=5, step=0.01, value=2, width=150)


@pn.depends(log_k_p_slider.param.value, 
            log_k_1_agon_slider.param.value, 
            log_k_1_endo_slider.param.value, 
            log_M_agon_slider.param.value, 
            log_M_endo_slider.param.value)
def fm_plotter(log_k_p, log_k_1_agon, log_k_1_endo, log_M_agon, log_M_endo):
    k_p = np.power(10.0, log_k_p)
    k_1_agon = np.power(10.0, log_k_1_agon)
    k_1_endo = np.power(10.0, log_k_1_endo)        
    M_agon = np.power(10.0, log_M_agon)
    M_endo = np.power(10.0, log_M_endo) 
    
    alpha_agon = 1/(1 + k_1_agon/k_p)
    alpha_endo = 1/(1 + k_1_endo/k_p)
    kappa_agon = (1-alpha_agon)*(Kd_agon + k_p/k_on)
    kappa_endo = (1-alpha_endo)*(Kd_endo + k_p/k_on)
    
    # steady state
    n_range = np.arange(1, 100)
    fractions = np.empty((2, len(n_range)))

    for n in n_range:
        Cn_agon = (alpha_agon**n)/(1-alpha_agon) * M_agon*T/(Kd_agon + k_p/k_on)
        Cn_endo = (alpha_endo**n)/(1-alpha_endo) * M_endo*T/(Kd_endo + k_p/k_on)

        Ctot_agon = M_agon*T/(Kd_agon + k_p/k_on) / (1-alpha_agon)
        Ctot_endo = M_endo*T/(Kd_endo + k_p/k_on) / (1-alpha_endo)

        fm_agon = Cn_agon/(Ctot_agon+M_agon) 
        fm_endo = Cn_endo/(Ctot_endo+M_endo)

        fractions[0, n-1] = fm_agon
        fractions[1, n-1] = fm_endo  
        
    p = bokeh.plotting.figure(
            height=400, 
            width=400, 
            title="𝑓𝑀 vs. 𝑛", 
            x_axis_label="n", 
            y_axis_label="fraction M activated", 
            y_range=(-0.1, 1.3) 
        )
    q = bokeh.plotting.figure(
            height=400, 
            width=400, 
            title="ratio agon:endo 𝑓𝑀 vs. 𝑛", 
            x_axis_label="n", 
            y_axis_label="ratio of fraction",
            y_range=(0,4)
        )

    p.circle(n_range, fractions[0], legend_label="agonist", color=red, fill_alpha=0.1, line_width=2.5, size=2)
    p.line(n_range, fractions[0], legend_label="agonist", line_color=red, line_width=1)
    
    p.circle(n_range, fractions[1], legend_label="endogenous", color=green, fill_alpha=0.1, line_width=2.5, size=2)
    p.line(n_range, fractions[1], legend_label="endogenous", line_color=green, line_width=1)
    
    q.circle(n_range, fractions[0]/fractions[1], color=blue) 
    q.line(n_range, fractions[0]/fractions[1], line_color=blue, line_width=1)
    
    if max(fractions[0]/fractions[1]) > 4:
        q.y_range=(0, max(fractions[0]/fractions[1]))
        
    return pn.Row(style(p), style(q))


lay_widgets = pn.Row(pn.Column(log_M_agon_slider, log_M_endo_slider), 
                     pn.Column(log_k_1_agon_slider, log_k_1_endo_slider), 
                     pn.Column(log_k_p_slider, align="center"), align="center")
dashboard_steady = pn.Column(lay_widgets, fm_plotter)
dashboard_steady

Column
    [0] Row(align='center')
        [0] Column
            [0] FloatSlider(name='agon log Mo', step=0.01, width=150)
            [1] FloatSlider(end=5, name='endo log Mo', start=1, step=0.01, value=2, width=150)
        [1] Column
            [0] FloatSlider(name='agon log k1', start=-3, step=0.01, value=-2, width=150)
            [1] FloatSlider(name='endo log k1', start=-3, step=0.01, value=-1, width=150)
        [2] Column(align='center')
            [0] FloatSlider(end=2, name='log kp', step=0.01, value=1, width=150)
    [1] ParamFunction(function)

In [16]:
# if the above does not work
dashboard_steady.show()

Launching server at http://localhost:49432


These plots are a bit bland, but I found them very informative. On the left we see the fraction of M that is activated, but it is hard to compare these visually since it is the relative ratios we care about, so that metric is plotted on the right. 
We have found a system where with increasing $n$, we get more selective (higher ratios with more steps). Toggling the agonist $k_{-1}$ brings the agonist selectivity much higher. 

**I've created a static graphic below to make my final observations more clear.**

In [13]:
palette_blue = [
     '#2a1c54',
     '#382670',
     '#46308d',
     '#483a99',
     '#4a45a5',
     '#4b4cad',
     '#4d53b5',
     '#5e64bc',
     '#7075c3',
     '#8286ca',
     '#9497d2',
     '#a5a8d9',
     '#b7bae1',
     '#c9cbe8',
     '#dbdcf0']
palette_red = [
     '#a10e47',
     '#b5134a',
     '#c7174e',
     '#d82454',
     '#e9355b',
     '#f34967',
     '#f85f75',
     '#fb7385',
     '#fc8698',
     '#fc99ac',
     '#fcadc2',
     '#fcbfd9',
     '#fdd1f0',
     '#fee1ff',
     '#ffebff']

M_agon = np.power(10.0, 0)
M_endo = np.power(10.0, 5) 

k_p = np.power(10.0, 1)
k_1_endo = np.power(10.0, -1)
k_1_agon_range = np.logspace(-2, 1, 20) # agonist fall-off rate is different

p = bokeh.plotting.figure(height=375, width=500, title="Varying agonist k_1: 𝑓𝑀 vs. 𝑛", 
                          x_axis_label="n", y_axis_label="fraction M activated", 
                          y_range=(-0.1, 1.3))
q = bokeh.plotting.figure(height=375, width=500, title="ratio agon:endo 𝑓𝑀 vs. 𝑛", 
                          x_axis_label="n", y_axis_label="ratio of fraction",
                          y_range=(-0.2,3)
                         )
for k_1_agon, _red, _blue in zip(k_1_agon_range, palette_red, palette_blue):
    alpha_agon = 1/(1 + k_1_agon/k_p)
    alpha_endo = 1/(1 + k_1_endo/k_p)
    kappa_agon = (1-alpha_agon)*(Kd_agon + k_p/k_on)
    kappa_endo = (1-alpha_endo)*(Kd_endo + k_p/k_on)

    n_range = np.arange(1, 100)
    fractions = np.empty((2, len(n_range)))

    for n in n_range:
        Cn_agon = (alpha_agon**n)/(1-alpha_agon) * M_agon*T/(Kd_agon + k_p/k_on)
        Cn_endo = (alpha_endo**n)/(1-alpha_endo) * M_endo*T/(Kd_endo + k_p/k_on)

        Ctot_agon = M_agon*T/(Kd_agon + k_p/k_on) / (1-alpha_agon)
        Ctot_endo = M_endo*T/(Kd_endo + k_p/k_on) / (1-alpha_endo)

        fm_agon = Cn_agon/(Ctot_agon+M_agon) 
        fm_endo = Cn_endo/(Ctot_endo+M_endo)

        fractions[0, n-1] = fm_agon
        fractions[1, n-1] = fm_endo  
        
    p.circle(n_range, fractions[0], color=_red, fill_alpha=0.1, line_width=1, size=1)
    p.line(n_range, fractions[0], line_color=_red, line_width=2.6)
    
    p.circle(n_range, fractions[1], legend_label="endogenous", 
             color=green, fill_alpha=0.1, line_color="white", line_width=1, size=5)
    p.line(n_range, fractions[1], legend_label="endogenous", 
             line_color=green, line_width=4)
    
    q.circle(n_range, fractions[0]/fractions[1], color=_blue, size=1) 
    q.line(n_range, fractions[0]/fractions[1], line_color=_blue, line_width=2.6)

for _palette, _plot, _direction in zip([palette_red, palette_blue],[p, q], ["left", "right"]):
    items = [ ( f"k_1={np.round(k_1_agon,2)}", 
              [_plot.circle(color=f"{_color}" ), 
            ]) 
         for k_1_agon, _color in zip(k_1_agon_range[::2], _palette[::2])]
    legend = bokeh.models.Legend(items=items, location="center")
    _plot.add_layout(legend, _direction)
    
layout = bokeh.layouts.layout([[style(p), style(q)]])
bokeh.io.show(layout)

We fix our endogenous $k_{-1}$ and see how to drive up agonist selectivity even when endogenous concentrations are 10,000 times higher. As expected, lower agonist $k_{-1}$'s lead to more selective regimes (red curve higher than green on the left plot, and curves sitting above 1 on the right plot). 

However, I am still not satisfied. Looking closely at the right plot, we see that an absurdly high $n$ is required to drive up fractional selectivity, while $k_{-1}$ reaches its limit of effectiveness. I wish I had more time, that's all for this week!

a poem, for the reader: 

### 3:02 AM 
Farewell my Agonist,  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Forever the protagonist  

Sincerely,   
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Mr. Endogenous—  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; the nitrogenous,   
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; androgynous,   
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; anonymous,  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; hippopotamus. 

# 🦛🦛🦛